# Elevation Comparison for AGU 2025

## CO_WestCentral_2019

Use 3DEP 1m seamless product + STV products

Approach: reproject / regrid other datasets to match reference lidar (in this case 3DEP seamless)

In [ ]:
# NOTE: unfortunately depends_on_optional machinery in a submodule prevents autoreload from working for that submodule
%load_ext autoreload
%autoreload 2

In [ ]:
import common_functions
import os

# TODO: consider moving some pieces into coincident library
import cql2
import numpy as np
import pyproj
import rasterio
import xarray as xr
from osgeo import gdal
from shapely.geometry import box
import geopandas as gpd
import matplotlib.pyplot as plt

import coincident
from coincident import pcd_fixtures

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Overview plots 

In [ ]:
site = "CO_WestCentral_2019"

site_meta = pcd_fixtures.read_pcd_site(site)

# Original search range around ALS dates, and granule names
site_search_results = pcd_fixtures.PCD_SITES[site]
site_search_results

In [ ]:
# common_functions.interactive_site_map(site_meta, title=site)

In [ ]:
ax = common_functions.static_site_map(site_meta, title=f"{site} - Dataset Coverage")

In [ ]:
common_functions.plot_timeline(
    site_meta,
    site_search_results["datetime"],
    title=f"Data Availability for Site: {site}",
)

## Standard product comparison

As a baseline, we'll compare standard DEM products, IS2, and GEDI


In [ ]:
gf_als, gf_maxar, gf_is2, gf_gedi, gf_overlap = common_functions.load_geodataframes(
    site_meta
)

## Get 1m 3DEP Seamless DEM (reference)

In [ ]:
# Need to use cql2 for property filtering with 3dep-1m
cql2_json = cql2.parse_text("collection='CO_WestCentral_2019_A19'").to_json()
gf_1m = coincident.search.search(
    dataset="3dep-1m", filter=cql2_json, intersects=gf_overlap
)
gf_1m.head()

In [ ]:
# Because 3DEP 1m on UTM boundaries is duplicated, pick one
# Count unique UTM zones
zone_counts = gf_1m["proj:code"].value_counts()
print(f"UTM zone counts:\n{zone_counts}")

# Pick the zone with more tiles
primary_zone = zone_counts.idxmax()
print(f"\nPrimary zone: {primary_zone}")

# Filter to keep only the primary zone
gf_1m = gf_1m[gf_1m["proj:code"] == primary_zone].copy()
# print(f"\nFiltered to {len(gf_1m)} tiles in {primary_zone}")

## Download non-COG high-resolution data locally

In [ ]:
# Get list of URLs to moasic
url_list = gf_1m["assets"].apply(lambda assets: assets["elevation"]["href"]).tolist()
url_list.sort()
url_list

In [ ]:
# Our best best I think is to use coincident to download these tiles locally first (then they are cached unlike GDAL HTTP driver)
paths = coincident.io.download.download_files(url_list, output_dir="/tmp/")

# Transform NAD83 (2011) NAVD88 to UTM with explicit 7912 datum

In [ ]:
# Overwrite metadata CRS to use correct 3D CRS
# There is no EPSG code for this, we need a custom WKT/PROJJSON definition
correct_crs = coincident.io.proj.construct_custom_utm_crs(
    gf_1m["proj:code"].iloc[0], a_srs="EPSG:6318", geoid="GEOID18"
)
correct_crs

In [ ]:
# Load VRT from list of tiffs
# NOTE: this also crops to AOI and assigns corrected 3D CRS
vrt_path = coincident.io.gdal.create_vrt(
    paths,
    outputBounds=gf_overlap.to_crs(correct_crs).total_bounds,
    a_srs=correct_crs.to_wkt(),
    prepend_vsicurl=False,
    # Force cleanly tapped result by specifying resolution
    custom_res=1.0,
    # NOTE: if coarsening with custom_res, prepending /vsicurl/ is actually much better
    # because single overview might get fetched...
    # prepend_vsicurl=False
    # custom_res=10.0,
    # resolution='user',
    # prepend_vsicurl=True
)

In [ ]:
# Construct and verify correct transform to go from NAD83(2011) / UTM zone 13N + NAVD88(GEOID18) to
# ITRF2020 / UTM zone 13N ellipsoid heights
with gdal.Open(vrt_path) as ds:
    srs = ds.GetProjection()

utm_7912 = coincident.io.proj.construct_custom_utm_crs(primary_zone, a_srs="EPSG:7912")
transformer = coincident.io.proj.get_proj_transform(pyproj.CRS(srs), utm_7912)
print(transformer.to_proj4(pretty=True))

In [ ]:
# Only difference is final ellipsoid model (+ellps=WGS84 vs GRS80)
# utm_9754 = coincident.io.proj.construct_custom_utm_crs(primary_zone, a_srs="EPSG:9754")
# transformer = coincident.io.proj.get_proj_transform(pyproj.CRS(srs), utm_9754)
# print(transformer.to_proj4(pretty=True))

In [ ]:
# Using the above transform create a warped VRT to reproject upon loading
infile = vrt_path

warped_vrt_path = coincident.io.gdal.warp_with_pipeline(
    infile, srs, utm_7912, transformer.to_proj4()
)

In [ ]:
%%time

# Finally, load our reference lidar, reprojecting on the fly
# NOTE: GDAL_NUM_THREADS="ALL_CPUS" really helps with performance, but still takes ~30-60s
# CPL_DEBUG=True,

with rasterio.Env(
    GDAL_NUM_THREADS="ALL_CPUS",
    # GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR" # maybe helps with performance of reading shift grid? doesn't seem to matter...
):
    da_lidar = xr.open_dataarray(warped_vrt_path, engine="rasterio").squeeze().compute()
da_lidar

In [ ]:
da_lidar.nbytes / (1024**2)  # size in MB

In [ ]:
# Write a local copy if you want for later:
# da_lidar.rio.to_raster("/tmp/CO_WestCentral_2019_1m_UTM13N_ITRF2014.tif", tiled=True, compress='lzw')

In [ ]:
# reduce to 10m for plotting
# WARNING: rioxarry does not update geotransform after coarsening steps
da_lidar_10m = da_lidar.coarsen(dict(x=10, y=10), boundary="trim").mean()

In [ ]:
da_lidar_10m.rio.crs

In [ ]:
ax = coincident.plot.plot_dem(da_lidar_10m)
gf_overlap.to_crs(da_lidar_10m.rio.crs).plot(ax=ax, facecolor="none", edgecolor="cyan");

## Load additional datasets

In [ ]:
## Load altimeters
# Get GEDI
data_gedi = coincident.io.sliderule.subset_gedi02a(
    gf_gedi,
    gf_overlap,
    include_worldcover=True,
)

In [ ]:
# data_gedi.to_parquet("data_gedi.parquet") # Save for later if repeatedly working with data
data_gedi.head()

In [ ]:
# What if we do our own sampling
# gfG = coincident.io.sliderule.to_3d(data_gedi, z_col="elevation_lm")
# NOTE: nothing dynamic here... would only apply to if helmerts are present (so changing ITRF or going to static CRS etc.)
# t = pyproj.Transformer.from_crs(pyproj.CRS(gfG.crs), utm_7912, always_xy=True)
# t.to_proj4()

In [ ]:
# data_gedi_r = gfG.to_crs(utm_7912) # da_lidar.rio.crs?
data_gedi_r = data_gedi.to_crs(da_lidar.rio.crs)
# print(data_gedi_r.crs)  # prints as projjson
data_gedi_r.head()

## Sample raster at altimeter locations


NOTE: currently we treat altimeters as 'points' rather than footprints. We do not do zonal statistics on pixels within the ground footprint. We just do bilinear interpolation of surrounding pixels to the point.

In [ ]:
# Sample our raster at these points
result = coincident.plot.sample_dem_at_points(
    da_lidar,
    data_gedi_r,
    "elevation_lm",
)
result.head()

In [ ]:
# Get ICSAT-2
# AttributeError: 'GeoDataFrame' object has no attribute 'atl06_quality_summary'
# NOTE: much slower to sample 3DEP... switch to new endpoint / parquet sampling?
data_is2 = coincident.io.sliderule.subset_atl06(
    gf_is2,
    gf_overlap,
    include_worldcover=True,
)

In [ ]:
# Reproject IS2
data_is2_r = data_is2.to_crs(da_lidar.rio.crs)
data_is2_r.head()

## Load COP30 & resample to 10m resolution

In [ ]:
# Instead of using gf_overlap, buffer USGSDEM Bounds by a bit
bounds = da_lidar_10m.rio.bounds()
buffer_lidar_aoi = (
    gpd.GeoDataFrame(
        geometry=[box(*bounds)],
        crs=da_lidar_10m.rio.crs,
    )
    .buffer(500)
    .to_crs("EPSG:4326")
)

In [ ]:
da_cop = coincident.io.xarray.load_dem_7912("cop30", aoi=buffer_lidar_aoi)
da_cop_10m = da_cop.rio.reproject_match(
    da_lidar_10m,
    resampling="bilinear",
).where(da_lidar_10m.notnull())

In [ ]:
dems = {"3DEP DTM (1m)": da_lidar_10m, "COP30": da_cop_10m}
# dems = {"3DEP DTM@10m": da_lidar_10m}
altimeters = {"ICESat-2": (data_is2_r, "h_li"), "GEDI": (data_gedi_r, "elevation_lm")}

In [ ]:
# UnboundLocalError: cannot access local variable 'diff_mappable' where it is not associated with a value
suptitle = f"Elevation Measurements - Reference LiDAR for {site} (10m posting)"

ax_dict = coincident.plot.compare_dems(
    dems,
    altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    # altimetry_basemap='hillshade',
    # elevation_clim=(2500, 4000),
    elevation_cmap="plasma",
    # NOTE: best size depends on aspect ratios and number of columns
    figsize=(8.5, 11),
    suptitle=suptitle,
)

## Zoom in for full resolution analysis

Let's look at where the majority of GEDI points are

In [ ]:
data_gedi.track.value_counts()

In [ ]:
# Drop single point from track 2362
data_gedi_r = data_gedi_r[data_gedi_r.track != 2362]
data_gedi_r.track.value_counts()

In [ ]:
ax = coincident.plot.plot_dem(da_lidar_10m)
# One little point from separate track...
gedi_points_valid_hull = gpd.GeoDataFrame(
    geometry=[data_gedi_r.union_all().convex_hull], crs=data_gedi_r.crs
)
gedi_points_valid_hull.plot(ax=ax, facecolor="none", edgecolor="red")

In [ ]:
# NOTE: gf.explore or holoviews (.hvplot) would be better for interactive plotting!
coincident.plot.plot_altimeter_points(
    data_gedi_r, "elevation_lm", basemap="Esri.WorldImagery", basemap_attribution=False
);

In [ ]:
# Zoom into a particular area and plot at full resolution
# 1. Clip by bounding box
# dem_subset = da_lidar.rio.clip_box(minx=3.43e5, miny=4.31e6, maxx=3.6e5, maxy=4.32e6)
# 2. Clip by geometry (GEDI convex hull) useful to hone in on specific dataset

lidar_subset = da_lidar.rio.clip(
    [gedi_points_valid_hull.geometry[0]]
)  # for some reason doesn't operate on geodataframe directly ...
# cop_subset = da_cop_10m
# Does it make sense to coarsen from 30m to 1m for COP?

da_hillshade = coincident.io.gdal.gdaldem(
    lidar_subset,
    "hillshade",
)
lidar_subset

In [ ]:
# NOTE: Why is top panel showing lower resolution tile zoom ?!
# Has to do with slight difference in extents if not first cropping to valid DEM extent

ax_dict = coincident.plot.compare_dems(
    {"3DEP DTM": lidar_subset},
    altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    elevation_cmap="plasma",
    figsize=(8.5, 11),
    altimetry_pointsize=10.0,
    suptitle=suptitle,
)

In [ ]:
# Crop altimeters before plotting
data_is2_r_crop = data_is2_r.clip(gedi_points_valid_hull.geometry[0])
data_gedi_r_crop = data_gedi_r.clip(gedi_points_valid_hull.geometry[0])

In [ ]:
altimeters = {
    "IS2 (h_li)": (data_is2_r_crop, "h_li"),
    "GEDI (lm)": (data_gedi_r_crop, "elevation_lm"),
}
# NOTE: could crop / mask worldcover and altimetry plots w/ lidar mask to really focus in on relevant area...
ax_dict = coincident.plot.compare_dems(
    {"3DEP DTM 1m": lidar_subset},
    altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    elevation_cmap="plasma",
    figsize=(8.5, 11),
    altimetry_pointsize=10.0,
    suptitle=suptitle,
)

## STV-generated products


1-meter posting generated with https://github.com/uw-cryo/lidar_tools

In [ ]:
# Requires `stv-user` credentials
# print(os.environ.get("AWS_PROFILE"))
!aws s3 ls --human-readable s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/

In [ ]:
!aws s3 ls --human-readable s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/stereo_WV01WV01_20190930_102001008BD60800_1020010088168700/

In [ ]:
# Not sure what to do with these files
#!aws s3 ls --human-readable s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/stereo_WV01WV01_20190930_102001008BD60800_1020010088168700/filled/

In [ ]:
# NOTE: CRS info is stored in sidecar .aux.xml so you can't do GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR
# But you *can* do CPL_VSIL_CURL_ALLOWED_EXTENSIONS which disables full directory scan
!CPL_DEBUG=OFF CPL_VSIL_CURL_ALLOWED_EXTENSIONS='.tif,.xml' AWS_PROFILE=stv-user gdalinfo /vsis3/uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/CO_WestCentral_2019-DSM_mos.tif

In [ ]:
HREF = "/vsis3/uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/stereo_WV01WV01_20190930_102001008BD60800_1020010088168700/20190930_2055_102001008BD60800_1020010088168700_1.0m-DEM_trans_dx+1.07m_dy+1.31m_dz+0.94m.tif"
!CPL_DEBUG=OFF CPL_VSIL_CURL_ALLOWED_EXTENSIONS='.tif,.xml' AWS_PROFILE=stv-user gdalinfo {HREF}

In [ ]:
STV_BUCKET_ENV = rasterio.Env(
    AWS_PROFILE="stv-user",
    CPL_VSIL_CURL_ALLOWED_EXTENSIONS=".tif,.xml",
)


def load_stv_product(href, overview_level=None):
    with STV_BUCKET_ENV:
        da = xr.open_dataarray(
            href,
            engine="rasterio",
            mask_and_scale=True,
            backend_kwargs={"open_kwargs": {"overview_level": overview_level}},
        ).squeeze()
    return da

In [ ]:
# Same comparison panel with custom lidar and stereo
# import coincident
# import rasterio
# import xarray as xr
prefix = "s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/"
dsm_href = prefix + "CO_WestCentral_2019-DSM_mos.tif"
dtm_href = prefix + "CO_WestCentral_2019-DTM_fill_window_size_4_mos.tif"
intensity_href = prefix + "CO_WestCentral_2019-intensity_mos.tif"

STV_BUCKET_ENV = rasterio.Env(
    AWS_PROFILE="stv-user",
    CPL_VSIL_CURL_ALLOWED_EXTENSIONS=".tif,.xml",
)


OVR = 2  # NOTE: -1:1m, 0:2m, 1:4m, 2:8m, 3:16m, etc
da_dtm = load_stv_product(dtm_href, overview_level=OVR)
da_dtm.name = "DTM"
da_dtm.rio.transform()

In [ ]:
da_dsm = load_stv_product(dsm_href, overview_level=OVR)
da_dsm.name = "DSM"
da_dsm

In [ ]:
# WHat pipeline is run for WGS84 (G2139) ~ ITRF2014?
# noop!
# So if we re-project STV DTM to USGS 1m seamless, we're just matching the grids.
coincident.io.proj.get_proj_transform(utm_7912, da_lidar.rio.crs)

In [ ]:
# Compare STV DTM against standard Lidar product DTM at overview level3
# NOTE: assume WGS84 (G2139) ~ ITRF2014
da_dtm_r = da_dtm.rio.reproject_match(da_lidar_10m, resampling="bilinear")

In [ ]:
# NOTE: I think something may be odd with the overviews
dems = {"3DEP DTM (10m)": da_lidar_10m, "STV DTM (10m)": da_dtm_r}
ax_dict = coincident.plot.compare_dems(
    dems,
    add_hillshade=True,
    diff_clim=(-5, 5),
)

### Intensity raster

In [ ]:
da_intensity = load_stv_product(intensity_href, overview_level=OVR)
da_intensity.name = "intensity"

In [ ]:
fig, ax = plt.subplots()
da_intensity.squeeze().plot.imshow(ax=ax, cmap="gray")
ax.set_aspect(1)
plt.title(intensity_href.split("/")[-1]);

### Stereo

In [ ]:
prefix = "s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/stereo_WV01WV01_20190930_102001008BD60800_1020010088168700"
stereo_href = f"{prefix}/20190930_2055_102001008BD60800_1020010088168700_1.0m-DEM_trans_dx+1.07m_dy+1.31m_dz+0.94m.tif"

da_stereo = load_stv_product(stereo_href, overview_level=OVR)
da_stereo.name = "stereo"

In [ ]:
da_stereo

In [ ]:
# Add Stereo DEM to the panel plot
suptitle = f"STV PCD Products for {site} (overview level 3)"
dems = {"STV 3DEP DTM": da_dtm, "STV 3DEP DSM": da_dsm, "STV Stereo DEM": da_stereo}
axes = coincident.plot.compare_dems(dems, suptitle=suptitle)

### Full resolution comparison


Try to keep images to less than 10kx10k for plotting

In [ ]:
# Just load 1m DEM for northern region
# Don't need to reproject_match since I think that's just a no-op?
da_dtm_crop = load_stv_product(dtm_href).rio.clip_box(*lidar_subset.rio.bounds())
da_dsm_crop = load_stv_product(dsm_href).rio.clip_box(*lidar_subset.rio.bounds())
da_stereo_crop = load_stv_product(stereo_href).rio.clip_box(*lidar_subset.rio.bounds())

da_dtm_crop

In [ ]:
# Ensure we're on the same grid
xr.testing.assert_equal(da_dtm_crop.x, lidar_subset.x)
xr.testing.assert_equal(da_dtm_crop.y, lidar_subset.y)

In [ ]:
# Just fake
# TODO: change coincident code to warn rather than error
for da in [da_dtm_crop, da_dsm_crop, da_stereo_crop]:
    da.rio.write_crs(lidar_subset.rio.crs, inplace=True)

da_dtm_crop.rio.crs

In [ ]:
dems = {
    "3DEP DTM 1m": lidar_subset,
    "PCD DTM 1m": da_dtm_crop,
    "PCD DSM 1m": da_dsm_crop,
    "PCD Stereo DEM 1m": da_stereo_crop,
}

# altimeters = {"IS2 (h_li)": (data_is2_r_crop , "h_li"), "GEDI (lm)": (data_gedi_r_crop, "elevation_lm")}

ax_dict = coincident.plot.compare_dems(
    dems,
    # altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    elevation_cmap="plasma",
    diff_clim=(-1, 1),
    figsize=(11, 8.5),  # landscape single page for lots of DTMs
    altimetry_pointsize=10.0,
    suptitle=suptitle,
)

In [ ]:
# More detailed histograms
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

diff_dtm = (lidar_subset - da_dtm_crop).to_series()
diff_dsm = (lidar_subset - da_dsm_crop).to_series()
diff_stereo = (lidar_subset - da_stereo_crop).to_series()

# Plot DTM difference histogram
coincident.plot.plot_diff_hist(diff_dtm, range=(-1,1), ax=axes[0])
axes[0].set_title('3DEP 1m DTM - PCD 1m DTM')

# Plot DSM difference histogram
coincident.plot.plot_diff_hist(diff_dsm, range=(-1,1), ax=axes[1])
axes[1].set_title('3DEP 1m DTM - PCD 1m DSM')

# Plot Stereo difference histogram
coincident.plot.plot_diff_hist(diff_stereo, range=(-1,1), ax=axes[2])
axes[2].set_title('3DEP 1m DTM - PCD 1m Stereo DEM')

plt.tight_layout()
plt.suptitle(f'{site}: Elevation Difference Histograms', y=1.02, fontsize=14)

### Altimeter differences

In [ ]:
# Look at altimeter difference
diff = coincident.plot.utils.sample_dem_at_points(da_dtm_crop, data_is2_r_crop, "h_li")
# diff.plot.scatter(x='x', y='y', c='elev_diff', cmap='bwr', vmin=-2,
coincident.plot.plot_diff_hist(diff.elev_diff, range=(-8, 8))
plt.title("IS2 h_li - PCD DTM 1m");

In [ ]:
# Look at altimeter difference
diff = coincident.plot.utils.sample_dem_at_points(da_dtm_crop, data_is2_r_crop, "h_li")
# diff.plot.scatter(x='x', y='y', c='elev_diff', cmap='bwr', vmin=-2,
coincident.plot.plot_diff_hist(diff.elev_diff, range=(-2, 2))
plt.title("IS2 h_li - PCD DTM 1m");

In [ ]:
diff = coincident.plot.utils.sample_dem_at_points(
    da_dtm_crop, data_gedi_r_crop, "elevation_lm"
)
# diff.plot.scatter(x='x', y='y', c='elev_diff', cmap='bwr', vmin=-2,
coincident.plot.plot_diff_hist(diff.elev_diff, range=(-8, 8))
plt.title("GEDI elevation_lm - PCD DTM 1m");

In [ ]:
# Add altimeters!
# "3DEP DTM 1m": lidar_subset,
suptitle = f"STV PCD Products and Altimetry for {site} (1m resolution)"

dems = {
    "PCD DTM": da_dtm_crop,
    "PCD DSM": da_dsm_crop,
    "PCD Stereo DEM": da_stereo_crop,
}
altimeters = {
    "IS2 (h_li)": (data_is2_r_crop, "h_li"),
    "GEDI (lm)": (data_gedi_r_crop, "elevation_lm"),
}

ax_dict = coincident.plot.compare_dems(
    dems,
    altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    elevation_cmap="plasma",
    diff_clim=(-10, 10),  # not great for DEMs, butter for al
    figsize=(11, 8.5),  # landscape single page for lots of DTMs
    altimetry_pointsize=10.0,
    suptitle=suptitle,
)